**参考资料：**
* [Transformer源码详解（Pytorch版本）逐行讲解](https://blog.csdn.net/Queen_sy/article/details/127628559?ops_request_misc=&request_id=&biz_id=102&utm_term=.view(batch_size,%20-1,%20n_heads,&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-2-127628559.142^v88^control_2,239^v2^insert_chatgpt&spm=1018.2226.3001.4187))

**模型架构：**

![transformer_model](model_picture.jpg "model")

In [22]:
import torch.nn as nn
import torch.optim as optim
import torch
import torch.utils.data as Data
import numpy as np

In [23]:
# Encoder_input    Decoder_input        Decoder_output
sentences = [['我 是 学 生 P', 'S I am a student', 'I am a student E'],         # S: 开始符号
             ['我 喜 欢 学 习', 'S I like learning P', 'I like learning P E'],  # E: 结束符号
             ['我 是 男 生 P', 'S I am a boy', 'I am a boy E']]                 # P: 占位符号，如果当前句子不足固定长度用P占位

src_vocab = {'P': 0, '我': 1, '是': 2, '学': 3, '生': 4, '喜': 5, '欢': 6, '习': 7, '男': 8}  # 词源字典  字：索引
src_idx2word = {src_vocab[key]: key for key in src_vocab}
src_vocab_size = len(src_vocab)  # 字典字的个数
tgt_vocab = {'P': 0, 'S': 1, 'E': 2, 'I': 3, 'am': 4, 'a': 5, 'student': 6, 'like': 7, 'learning': 8, 'boy': 9}
idx2word = {tgt_vocab[key]: key for key in tgt_vocab}   # 把目标字典转换成 索引：字的形式
tgt_vocab_size = len(tgt_vocab)                         # 目标字典尺寸
src_len = len(sentences[0][0].split(" "))               # Encoder输入的最大长度
tgt_len = len(sentences[0][1].split(" "))               # Decoder输入输出最大长度

print(src_vocab_size, src_len, tgt_vocab_size, tgt_len)

9 5 10 5


In [24]:
# 把sentences 转换成字典索引
def make_data():
    enc_inputs, dec_inputs, dec_outputs = [], [], []
    for i in range(len(sentences)):
        enc_input = [[src_vocab[n] for n in sentences[i][0].split()]]
        dec_input = [[tgt_vocab[n] for n in sentences[i][1].split()]]
        dec_output = [[tgt_vocab[n] for n in sentences[i][2].split()]]
        enc_inputs.extend(enc_input)
        dec_inputs.extend(dec_input)
        dec_outputs.extend(dec_output)
    return torch.LongTensor(enc_inputs), torch.LongTensor(dec_inputs), torch.LongTensor(dec_outputs)


# 自定义数据集函数
class MyDataSet(Data.Dataset):
    def __init__(self, enc_inputs, dec_inputs, dec_outputs):
        super(MyDataSet, self).__init__()
        self.enc_inputs = enc_inputs
        self.dec_inputs = dec_inputs
        self.dec_outputs = dec_outputs

    def __len__(self):
        return self.enc_inputs.shape[0]

    def __getitem__(self, idx):
        return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]

In [25]:
# 将句子主体进行编码，把sentences转换成字典索引
enc_inputs, dec_inputs, dec_outputs = make_data()
print(enc_inputs)
print(dec_inputs)
print(dec_outputs)

tensor([[1, 2, 3, 4, 0],
        [1, 5, 6, 3, 7],
        [1, 2, 8, 4, 0]])
tensor([[1, 3, 4, 5, 6],
        [1, 3, 7, 8, 0],
        [1, 3, 4, 5, 9]])
tensor([[3, 4, 5, 6, 2],
        [3, 7, 8, 0, 2],
        [3, 4, 5, 9, 2]])


In [26]:
# 自定义数据集函数
loader = Data.DataLoader(MyDataSet(enc_inputs, dec_inputs, dec_outputs), 2, True)
print(loader)

In [27]:
d_model = 512   # 字 Embedding 的维度
d_ff = 2048     # 前向传播隐藏层维度
d_k = d_v = 64  # K(=Q), V的维度
n_layers = 6    # 有多少个 encoder和 decoder
n_heads = 8     # Multi-Head Attention设置为8

In [28]:
# 位置嵌入模块，输入位置信息
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pos_table = np.array([
            [pos / np.power(10000, 2 * i / d_model) for i in range(d_model)]
            if pos != 0 else np.zeros(d_model) for pos in range(max_len)])
        
        pos_table[1:, 0::2] = np.sin(pos_table[1:, 0::2])           # 字嵌入维度为偶数时
        pos_table[1:, 1::2] = np.cos(pos_table[1:, 1::2])           # 字嵌入维度为奇数时

        self.pos_table = torch.FloatTensor(pos_table)        # enc_inputs: [seq_len, d_model]

    def forward(self, enc_inputs):                                  # enc_inputs: [batch_size, seq_len, d_model]
        enc_inputs += self.pos_table[:enc_inputs.size(1), :]      # 根据输入语句的长度进行相加
        return self.dropout(enc_inputs) # 随机丢弃


In [29]:
# 填充掩盖
def get_attn_pad_mask(seq_q, seq_k):                                # seq_q: [batch_size, seq_len] ,seq_k: [batch_size, seq_len]
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(0)判断数据是否是pad，如果是则为True
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)                   # 判断 输入那些含有P(=0),用1标记 ,[batch_size, 1, len_k] 
    return pad_attn_mask.expand(batch_size, len_q, len_k)           # 扩展成多维度

In [30]:
# 时序掩盖
def get_attn_subsequence_mask(seq):                                 # seq: [batch_size, tgt_len]
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1)            # 生成上三角矩阵,[batch_size, tgt_len, tgt_len]
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()    # [batch_size, tgt_len, tgt_len]
    return subsequence_mask

In [31]:
# 点积注意力模块
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):                              # Q: [batch_size, n_heads, len_q, d_k]
                                                                        # K: [batch_size, n_heads, len_k, d_k]
                                                                        # V: [batch_size, n_heads, len_v(=len_k), d_v]
                                                                        # attn_mask: [batch_size, n_heads, seq_len, seq_len]

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)    # scores : [batch_size, n_heads, len_q, len_k]
        # 把被mask的地方置为无限小，softmax之后基本就是0，即对pad的单词不起作用
        scores.masked_fill_(attn_mask, -1e9)                            # 如果时停用词P就等于 0
        
        attn = nn.Softmax(dim=-1)(scores)
        # 每个字符的embedding，即经过考虑其他所有字符影响的embedding
        context = torch.matmul(attn, V)                                 # [batch_size, n_heads, len_q, d_v]
        
        return context, attn

In [32]:
# 多头注意力模块
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)


    def forward(self, input_Q, input_K, input_V, attn_mask):    # input_Q: [batch_size, len_q, d_model]
                                                                # input_K: [batch_size, len_k, d_model]
                                                                # input_V: [batch_size, len_v(=len_k), d_model]
                                                                # attn_mask: [batch_size, seq_len, seq_len]
        residual, batch_size = input_Q, input_Q.size(0)
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)    # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)    # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1, 2)    # V: [batch_size, n_heads, len_v(=len_k), d_v]
        
        
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)                 # attn_mask : [batch_size, n_heads, len_q, len_k]

        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)             # context: [batch_size, n_heads, len_q, d_v]
                                                                                    # attn: [batch_size, n_heads, len_q, len_k]

        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v)    # context: [batch_size, len_q, n_heads * d_v]

        output = self.fc(context)                                                   # [batch_size, len_q, d_model]
        
        return nn.LayerNorm(d_model)(output + residual), attn      # output = [batch_size, len_q, d_model]; attn = [batch_size, n_heads, len_q, len_k]



In [33]:
# 前馈神经网络
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=True),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=True))

    def forward(self, inputs):                                  # inputs: [batch_size, seq_len, d_model]
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model)(output + residual)  # [batch_size, seq_len, d_model]

In [34]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()                   # 多头注意力机制
        self.pos_ffn = PoswiseFeedForwardNet()                      # 前馈神经网络

    def forward(self, enc_inputs, enc_self_attn_mask):              # enc_inputs: [batch_size, src_len, d_model]
        # 输入3个enc_inputs分别与W_q、W_k、W_v相乘得到Q、K、V            # enc_self_attn_mask: [batch_size, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs,
                                                                    # enc_outputs: [batch_size, src_len, d_model],
                                               enc_self_attn_mask)  # attn: [batch_size, n_heads, src_len, src_len]
        
        enc_outputs = self.pos_ffn(enc_outputs)                     # enc_outputs: [batch_size, src_len, d_model]  # [batch_size, len_q, d_model]
        
        return enc_outputs, attn      # [batch_size, len_q, d_model]  ; attn = [batch_size, n_heads, len_q, len_k]

In [35]:
# 编码器
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(src_vocab_size, d_model)                     # 把字转换字向量
        self.pos_emb = PositionalEncoding(d_model)                               # 加入位置信息
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):                                               # enc_inputs: [batch_size, src_len]
        enc_outputs = self.src_emb(enc_inputs)                                   # enc_outputs: [batch_size, src_len, d_model] 2*5*512
        enc_outputs = self.pos_emb(enc_outputs)                                  # enc_outputs: [batch_size, src_len, d_model] 2*5*512
        
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)           # enc_self_attn_mask: [batch_size, src_len, src_len]
        enc_self_attns = []

        for layer in self.layers:
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)  # enc_outputs :   [batch_size, src_len, d_model],
                                                                                 # enc_self_attn : [batch_size, n_heads, src_len, src_len]
            enc_self_attns.append(enc_self_attn)
        
        return enc_outputs, enc_self_attns  # enc_outputs = [batch_size, len_q, d_model] ; src_len == len_q

In [36]:
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask,
                dec_enc_attn_mask):                                             # dec_inputs: [batch_size, tgt_len, d_model]
                                                                                # enc_outputs: [batch_size, src_len, d_model]
                                                                                # dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
                                                                                # dec_enc_attn_mask: [batch_size, tgt_len, src_len]
        # 自注意力
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs,
                                                        dec_inputs,
                                                        dec_self_attn_mask)     # dec_outputs: [batch_size, tgt_len, d_model]
                                                                                # dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        # encoder-decoder中间层
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs,
                                                      enc_outputs,
                                                      dec_enc_attn_mask)        # dec_outputs: [batch_size, tgt_len, d_model]
                                                                                # dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
        
        dec_outputs = self.pos_ffn(dec_outputs)                                 # dec_outputs: [batch_size, tgt_len, d_model]
        
        return dec_outputs, dec_self_attn, dec_enc_attn                         # dec_outputs: [batch_size, tgt_len, d_model]

In [37]:
# 解码器
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):                         # dec_inputs: [batch_size, tgt_len]
                                                                                    # enc_intpus: [batch_size, src_len]
                                                                                    # enc_outputs: [batsh_size, src_len, d_model]

        dec_outputs = self.tgt_emb(dec_inputs)                                      # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_outputs)                                     # [batch_size, tgt_len, d_model]
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs)          # [batch_size, tgt_len, tgt_len]

        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs)                       # [batch_size, tgt_len, tgt_len]
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask +dec_self_attn_subsequence_mask), 0)   # [batch_size, tgt_len, tgt_len]
        
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)               # [batc_size, tgt_len, src_len]
        
        dec_self_attns = []
        dec_enc_attns = []
        
        for layer in self.layers:                                                   # dec_outputs: [batch_size, tgt_len, d_model]
                                                                                    # dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
                                                                                    # dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]                                                                            
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask,
                                                             dec_enc_attn_mask)
            
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)

        return dec_outputs, dec_self_attns, dec_enc_attns

In [38]:
# 
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.Encoder = Encoder()
        self.Decoder = Decoder()
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False)

    def forward(self, enc_inputs, dec_inputs):                          # enc_inputs: [batch_size, src_len]
                                                                        # dec_inputs: [batch_size, tgt_len]
        # 编码器
        enc_outputs, enc_self_attns = self.Encoder(enc_inputs)          # enc_outputs: [batch_size, src_len, d_model] 编码器输出
                                                                        # enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len] 编码相似矩阵
        
        # 解码器
        dec_outputs, dec_self_attns, dec_enc_attns = self.Decoder(
            dec_inputs, enc_inputs, enc_outputs)                        # dec_outpus    : [batch_size, tgt_len, d_model],
                                                                        # dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len],
                                                                        # dec_enc_attn  : [n_layers, batch_size, tgt_len, src_len]
        
        # 预测
        dec_logits = self.projection(dec_outputs)                       # dec_logits: [batch_size, tgt_len, tgt_vocab_size]
       
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns


In [39]:
# 模型训练
model = Transformer()
criterion = nn.CrossEntropyLoss(ignore_index=0)         # 忽略 占位符 索引为0.
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)


for epoch in range(50):
    for enc_inputs, dec_inputs, dec_outputs in loader:  # enc_inputs : [batch_size, src_len]
                                                            # dec_inputs : [batch_size, tgt_len]
                                                            # dec_outputs: [batch_size, tgt_len]
        # print("enc_inputs：",enc_inputs)
        # print("dec_inputs：",dec_inputs)
        # print("dec_outputs：",dec_outputs)
        enc_inputs, dec_inputs, dec_outputs = enc_inputs, dec_inputs, dec_outputs

        
        outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
                                                            # outputs: [batch_size * tgt_len, tgt_vocab_size]
        
        loss = criterion(outputs, dec_outputs.view(-1))
        print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

torch.save(model, 'model.pth')
print("保存模型成功")


Epoch: 0001 loss = 2.231974
Epoch: 0001 loss = 2.504350
Epoch: 0002 loss = 1.775698
Epoch: 0002 loss = 1.537487
Epoch: 0003 loss = 1.350026
Epoch: 0003 loss = 1.281363
Epoch: 0004 loss = 1.014904
Epoch: 0004 loss = 1.044062
Epoch: 0005 loss = 0.608651
Epoch: 0005 loss = 0.687440
Epoch: 0006 loss = 0.495752
Epoch: 0006 loss = 0.213663
Epoch: 0007 loss = 0.257284
Epoch: 0007 loss = 0.355675
Epoch: 0008 loss = 0.269838
Epoch: 0008 loss = 0.070932
Epoch: 0009 loss = 0.163931
Epoch: 0009 loss = 0.032700
Epoch: 0010 loss = 0.099848
Epoch: 0010 loss = 0.100586
Epoch: 0011 loss = 0.126517
Epoch: 0011 loss = 0.017396
Epoch: 0012 loss = 0.102813
Epoch: 0012 loss = 0.044663
Epoch: 0013 loss = 0.030780
Epoch: 0013 loss = 0.108474
Epoch: 0014 loss = 0.032578
Epoch: 0014 loss = 0.103485
Epoch: 0015 loss = 0.033874
Epoch: 0015 loss = 0.066195
Epoch: 0016 loss = 0.017371
Epoch: 0016 loss = 0.096116
Epoch: 0017 loss = 0.049345
Epoch: 0017 loss = 0.015102
Epoch: 0018 loss = 0.009565
Epoch: 0018 loss = 0

In [40]:
def test(model, enc_input, start_symbol): # 神经网络模型，输入字符串，开始符
    enc_outputs, enc_self_attns = model.Encoder(enc_input) # 输入编码
    dec_input = torch.zeros(1, tgt_len).type_as(enc_input.data) # 解码器输入初始化
    next_symbol = start_symbol # 开始符
    for i in range(0, tgt_len): # 解码器输出最大长度
        dec_input[0][i] = next_symbol
        dec_outputs, _, _ = model.Decoder(dec_input, enc_input, enc_outputs)
        projected = model.projection(dec_outputs)
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
        next_word = prob.data[i]
        next_symbol = next_word.item()
    return dec_input # 输出结果

In [41]:
enc_inputs, dec_inputs, dec_outputs = make_data()
loader = Data.DataLoader(MyDataSet(enc_inputs, dec_inputs, dec_outputs), 2, True)
enc_inputs, _, _ = next(iter(loader))
model = torch.load('model.pth')
predict_dec_input = test(model, enc_inputs[0].view(1, -1), start_symbol = tgt_vocab["S"]) # 解码器输入
predict, _, _, _ = model(enc_inputs[0].view(1, -1), predict_dec_input) # 预测结果
predict = predict.data.max(1, keepdim=True)[1]

print([src_idx2word[int(i)] for i in enc_inputs[0]], '->',
      [idx2word[n.item()] for n in predict.squeeze()])

['我', '是', '男', '生', 'P'] -> ['I', 'am', 'a', 'boy', 'E']
